In [1]:
import sys
import os

# Agregar la ruta al proyecto para importar core desde security
sys.path.append("C:/Desarrollo/strix-system")

In [2]:
from security.services.health_check import health_check
status = health_check()
print(f"🔍 Estado de conexiones: {status}")

✅ Nueva Conexión a PostgreSQL establecida.
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy


2025-04-12 19:04:33,963 - INFO - ✅ PostgreSQL está operativo.
2025-04-12 19:04:34,644 - INFO - ✅ S3 está operativo.


✅ Conexión a S3 establecida con éxito.
🔍 Estado de conexiones: {'postgres': 'OK', 's3': 'OK'}


In [4]:
from security.data_access.user_repository import UserRepository
from security.utils.password_utils import hash_password, verify_password
from sqlalchemy.exc import SQLAlchemyError

def test_create_user():
    repo = UserRepository()
    try:
        print("🧪 Creando usuario de prueba...")
        password = "supersegura12345"
        hashed = hash_password(password)
        user = repo.create_user(
            username="testuser45",
            email="testuser5@example.com",
            full_name="Usuario de Prueba",
            password=hashed
        )
        assert user["id"] is not None
        print("✅ Usuario creado correctamente.")
        return user, hashed
    except SQLAlchemyError as e:
        print(f"❌ Error al crear usuario: {e}")
        return None, None

def test_get_user_by_username(username):
    repo = UserRepository()
    print(f"🔍 Buscando usuario por username: {username}...")
    user = repo.get_user_by_username(username)
    if user:
        print(f"✅ Usuario encontrado: {user['username']}")
    else:
        print("❌ Usuario no encontrado.")
    return user

def test_get_all_users():
    repo = UserRepository()
    print("📋 Listando todos los usuarios...")
    users = repo.get_all_users()
    print(f"👥 Total de usuarios: {len(users)}")
    return users

def test_deactivate_user(user_id):
    repo = UserRepository()
    print(f"🛑 Desactivando usuario con ID {user_id}...")
    user = repo.deactivate_user(user_id)
    if user and not user["is_active"]:
        print("✅ Usuario desactivado.")
    else:
        print("❌ Falló la desactivación del usuario.")

def test_password_verification(password,hashed_password):
    print("🔐 Verificando contraseña...")
    if verify_password(password, hashed_password):
        print("✅ Contraseña verificada correctamente.")
    else:
        print("❌ Verificación de contraseña fallida.")




In [5]:
created_user, hashed = test_create_user()


✅ Conexión a PostgreSQL establecida.
🧪 Creando usuario de prueba...
✅ Usuario creado correctamente.


In [8]:
password = "supersegura1234"

test_password_verification(password,hashed)


🔐 Verificando contraseña...
❌ Verificación de contraseña fallida.


In [9]:
created_user, hashed

({'id': 59,
  'username': 'testuser45',
  'email': 'testuser5@example.com',
  'full_name': 'Usuario de Prueba',
  'is_active': True,
  'created_at': '2025-04-10T01:12:57.578553'},
 '$2b$12$5fF.9gkYvgivW8F.A8d48OuqPlZxv79FPzLPAdAaaiOZuO4YVWtzG')

In [10]:
test_password_verification(password, hashed)


🔐 Verificando contraseña...
❌ Verificación de contraseña fallida.


In [11]:
test_get_user_by_username(created_user["username"])



🔍 Buscando usuario por username: testuser45...
✅ Usuario encontrado: testuser45


{'id': 59,
 'username': 'testuser45',
 'email': 'testuser5@example.com',
 'full_name': 'Usuario de Prueba',
 'is_active': True,
 'created_at': '2025-04-10T01:12:57.578553'}

In [12]:
test_get_all_users()


📋 Listando todos los usuarios...
👥 Total de usuarios: 2


[{'id': 43,
  'username': 'adminuser',
  'email': 'admin@example.com',
  'full_name': 'Admin User',
  'is_active': True,
  'created_at': '2025-04-09T10:16:12.904968'},
 {'id': 59,
  'username': 'testuser45',
  'email': 'testuser5@example.com',
  'full_name': 'Usuario de Prueba',
  'is_active': True,
  'created_at': '2025-04-10T01:12:57.578553'}]

In [13]:
test_deactivate_user(created_user["id"])

🛑 Desactivando usuario con ID 59...
✅ Usuario desactivado.


In [3]:
repo = UserRepository()
user = repo.validate_user_credentials("testuser4", "supersegura1234")
if user:
    print("✅ Login exitoso:", user)
else:
    print("❌ Usuario o contraseña inválidos.")

✅ Conexión a PostgreSQL establecida.
✅ Login exitoso: {'id': 11, 'username': 'testuser4', 'email': 'testuser4@example.com', 'full_name': 'Usuario de Prueba', 'is_active': False, 'created_at': '2025-04-08T18:44:33.624632'}


In [1]:
import sys
import os

# Agregar la ruta al proyecto para importar core desde security
sys.path.append("C:/Desarrollo/strix-system")

from security.data_access.user_repository import UserRepository

def test_validate_user_credentials():
    repo = UserRepository()

    # Estos datos deberían existir en tu DB para que este test funcione
    username = "adminuser"
    password = "adminpass"  # contraseña real que matchea con el hash en la DB

    result = repo.validate_user_credentials(username, password)

    assert result is not None, "Debe retornar datos si las credenciales son válidas"
    assert "user" in result, "Debe retornar los datos del usuario"
    assert "roles" in result, "Debe incluir los roles"
    assert isinstance(result["roles"], list), "Roles debe ser una lista"

    for role in result["roles"]:
        assert "id" in role and "name" in role
        assert "permissions" in role
        for perm in role["permissions"]:
            assert "id" in perm and "name" in perm




In [3]:
repo = UserRepository()

# Estos datos deberían existir en tu DB para que este test funcione
username = "adminuser"
password = "adminpass"  # contraseña real que matchea con el hash en la DB

result = repo.validate_user_credentials(username, password)

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper[User(users)]'. Original exception was: When initializing mapper Mapper[User(users)], expression 'Role' failed to locate a name ('Role'). If this is a class name, consider adding this relationship() to the <class 'security.models.user.User'> class after both dependent classes have been defined.

In [2]:
test_validate_user_credentials()

✅ Conexión a PostgreSQL establecida.


InvalidRequestError: When initializing mapper Mapper[User(users)], expression 'Role' failed to locate a name ('Role'). If this is a class name, consider adding this relationship() to the <class 'security.models.user.User'> class after both dependent classes have been defined.